In [1]:
import os
os.environ["XLA_FLAGS"]="--xla_gpu_force_compilation_parallelism=1"
import stable_baselines3
from stable_baselines3 import PPO
from object_env import ObjectEnv
from predict_transformer_nonlinear import NonLinearNet
import torch
import numpy as np
import wandb

/home/wschwarzer/anaconda3/envs/vision2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# PPO.load("transformer-irl-network-training/imitation_agents_dummy/[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037 -0.33200289 -0.14191921  0.34818161  0.01508194]/bc/170.80142617225647")

# "[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037 -0.33200289 -0.14191921  0.34818161  0.01508194]/bc/169.92737138271332"

# os.path.exists("[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037 -0.33200289 -0.14191921  0.34818161  0.01508194]/bc/169.92737138271332")

# os.listdir()

# os.path.exists('[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037\n -0.33200289 -0.14191921  0.34818161  0.01508194]/bc/169.92737138271332')

# PPO.load('[ 0.02168476  0.43229203  0.32464645  0.54754327  0.1412109   0.21547867\n  0.13438869 -0.47121423  0.31777201 -0.00078345]/bc/173.823211312294')

In [3]:
pi = PPO.load('imitation_agents/[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037\n -0.33200289 -0.14191921  0.34818161  0.01508194]/sirl/10.257024141825182')

In [4]:
cur_weights = np.array([ 0.25186944, -0.32851523, -0.32239092,  0.47484801,  0.21261153, -0.44966037, -0.33200289, -0.14191921,  0.34818161,  0.01508194])

In [5]:
wandb.init(project="il_viz")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: willschwarzer. Use `wandb login --relogin` to force relogin


In [6]:
# load in traj_reps and weights from weights_and_reps
traj_reps, weights = np.load("weights_and_reps/traj_reps.npy"), np.load("weights_and_reps/weights.npy")

In [7]:
# Check the first weight and traj_rep
print(weights[0])
print(traj_reps[0])

[ 0.25186944 -0.32851523 -0.32239092  0.47484801  0.21261153 -0.44966037
 -0.33200289 -0.14191921  0.34818161  0.01508194]
[-1.01610114e+03 -3.73619659e+02  1.50294714e+03 -1.48694507e+03
  1.28654333e+03 -2.06587540e+02  2.10560107e+03 -1.26355682e+02
 -5.87806335e+02  8.01946167e+02 -7.68933960e+02  1.25030908e+03
 -7.34751648e+02  8.98366623e+01  6.99628418e+02 -1.14113319e+02
  2.47940002e+02  3.83757721e+02  3.70705719e+02 -2.20221558e+02
 -9.30197693e+02 -5.08494385e+02 -4.01086578e+02  5.77153809e+02
  7.24062256e+02  9.83276672e+02 -1.92496674e+02 -1.31703320e+03
  5.58188965e+02 -1.48397510e+03  5.62899536e+02  4.15922585e+01
 -5.91537170e+01  9.72961121e+02  1.14650696e+03 -6.37975633e-01
 -5.83768494e+02 -1.01312384e+03  9.51759644e+02 -5.37287788e+01
  1.04304504e+03  8.93709167e+02  7.97610168e+02 -8.76514832e+02
  1.28007751e+02  1.03961353e+03 -1.72000961e+01 -1.12210034e+03
 -4.35583588e+02  8.93122375e+02 -1.42542603e+03 -2.25862891e+03
 -2.76652405e+02  7.40386047e+01

In [8]:
def load_model(location, obs_size, horizon):
    trajectory_rep_dim = 100
    state_rep_dim = 100
    state_hidden_size = 2048
    num_state_layers = 2
    trajectory_hidden_size = 2048
    num_trajectory_layers = 2
    mlp = False
    trajectory_sigmoid = False
    lstm = False
    ground_truth_phi = False
    
    net = NonLinearNet(trajectory_rep_dim, 
                       state_rep_dim, 
                       state_hidden_size, 
                       64, 
                       trajectory_hidden_size, 
                       obs_size, 
                       horizon, 
                       num_trajectory_layers, 
                       num_state_layers, 
                       mlp=mlp, 
                       trajectory_sigmoid=trajectory_sigmoid, 
                       lstm=lstm, 
                       ground_truth_phi = ground_truth_phi)
    # net = net.cuda() # Maybe shouldn't make this cuda
    net.load_state_dict(torch.load(location))
    return net

In [9]:
# Load in state encoder from load_model with location=nshot_2.parameters, obs_size=10, horizon=50
state_encoder = load_model("nshot_2.parameters", 10, 50).state_encoder

In [10]:
env = ObjectEnv(traj_reps[0], 
                num_rings=5, 
                env_size=1, 
                move_allowance=True, 
                episode_len=50, 
                min_block_dist=0.25, 
                intersection=True, 
                max_move_dist=0.1, 
                block_thickness=2, 
                single_move=False,
                ground_truth_object_rewards=weights[0],
                state_encoder=state_encoder)

In [11]:
ims = env.rollout_and_render(pi, True)

[[ 1.521497    0.6635676 ]
 [-1.1961111   0.54925245]
 [ 0.5015235  -0.7241829 ]
 [-0.05469772  1.1380029 ]
 [-0.08984672 -0.19588546]]
0.019187202285975275
[[ 0.02919327  0.01273201]
 [-0.02295003  0.01053862]
 [ 0.00962283 -0.01389504]
 [-0.0010495   0.02183509]
 [-0.00172391 -0.00375849]]
0.1 5.2118073
[[ 0.29172504  0.02889492]
 [ 0.06804456  0.42559664]
 [-0.42896394 -0.4128707 ]
 [-0.4797816   0.33261985]
 [ 0.27815675  0.37001215]]
[[ 0.32091831  0.04162693]
 [ 0.04509454  0.43613526]
 [-0.41934111 -0.42676574]
 [-0.4808311   0.35445494]
 [ 0.27643284  0.36625365]]
[[-2.076237    0.5607748 ]
 [-0.63580495 -0.39806047]
 [ 0.14941393  0.30706444]
 [-0.69955623 -0.16408505]
 [-0.3554501   0.2686475 ]]
0.022694529532240888
[[-0.04711922  0.01272652]
 [-0.01442929 -0.0090338 ]
 [ 0.00339088  0.00696868]
 [-0.0158761  -0.00372383]
 [-0.00806677  0.00609683]]
0.1 4.406348
[[ 0.32091831  0.04162693]
 [ 0.04509454  0.43613526]
 [-0.41934111 -0.42676574]
 [-0.4808311   0.35445494]
 [ 0.27

In [12]:
wandb.log({"video": wandb.Video(ims, fps=4, format="mp4")})